# Supplementing the Schaefer atlases with Subcortical Structures

in `JointAtlas.ipynb` and `VolumetricMerging.ipynb` we downloaded and combined some subcortical atlases. Now we actually want to combine them with the volumetric Schaefer atlases to get 4S in volume space. 

In [1]:
import os
import nibabel as nb

tfhome = os.getenv("TEMPLATEFLOW_HOME")

nlin6_subcortical = "tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-01_dseg.nii.gz"
nlin6_subcortical_tsv = "tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-01_dseg.tsv"
nlin6_subcortical_img = nb.load(nlin6_subcortical)

nlin09c_subcortical = (
    "tpl-MNI152NLin2009cAsym_atlas-SubcorticalMerged_res-01_dseg.nii.gz"
)
nlin09c_subcortical_tsv = (
    "tpl-MNI152NLin2009cAsym_atlas-SubcorticalMerged_res-01_dseg.tsv"
)
nlin09c_subcortical_img = nb.load(nlin09c_subcortical)

In [2]:
from glob import glob
import numpy as np


def check_grids(atlas_files):
    print("checking files:")
    print("\n  " + "\n  ".join(atlas_files))

    print("\ncomparing grids:")
    ref_img = nb.load(atlas_files.pop())

    for try_file in atlas_files:
        try_img = nb.load(try_file)
        if not np.allclose(try_img.affine, ref_img.affine):
            raise Exception(
                "incompatible affines:", try_img, try_img.affine, ref_img.affine
            )

        if not try_img.shape == ref_img.shape:
            raise Exception(
                "incompatible shapes:", try_img, try_img.shape, ref_img.shape
            )

        print(try_img.shape, "==", ref_img.shape)

    print("All grids match orientation and shape!!")


nlin6_schaefers = glob(
    "Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-*dseg.nii.gz"
)
nlin09c_schaefers = glob(
    "Schaefer/tpl-MNI152NLin2009cAsym_atlas-Schaefer2018v0143_res-01_desc-*dseg.nii.gz"
)

check_grids([nlin6_subcortical] + nlin6_schaefers)
check_grids([nlin09c_subcortical] + nlin09c_schaefers)

checking files:

  tpl-MNI152NLin6Asym_atlas-SubcorticalMerged_res-01_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-800ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-500ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-300ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-600ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-700ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-200ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-900ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-400ParcelsAllNetworks_dseg.nii.gz
  Schaefer/tpl-MNI152NLin6Asym_atlas-Schaefer2018v0143_res-01_desc-100ParcelsAllNetworks_dseg.nii.gz
  Schaefe

Let's look at what one of these entries looks like:

In [3]:
import numpy as np
import pandas as pd


def roi_data(nii_file):
    return nb.load(nii_file).get_fdata().astype(np.uint32)


def get_region_data(tsv_file):
    df = pd.read_csv(tsv_file, sep="\t")
    return df["index"].tolist(), df["label"].tolist()


def tsv_to_config(tsv_file):
    config = {}
    config["node_ids"], config["node_names"] = get_region_data(tsv_file)
    return config


def verify_atlas(atlas_config, atlas_data):
    data_regions = np.unique(atlas_data)
    data_regions = data_regions[data_regions > 0].tolist()
    for node_id, node_name in zip(atlas_config["node_ids"], atlas_config["node_names"]):
        if node_id not in data_regions:
            raise Exception(f"{node_id}: {node_name} not in atlas data")

    missing_regions = set(atlas_config["node_ids"]).difference(data_regions)
    if missing_regions:
        raise Exception(f"{missing_regions} present in data but not in labels")

## Combining the atlases

We need to add the cerebellum, thalamus and subcortical regions into a single atlas. Below are some functions that help us add one atlas into another. We'll add these three together and then add them to all the other atlases that don't have any subcortical regions

In [4]:
from scipy.stats import mode


def match_partitions(data1, data2):
    """Match the regions in data1 to the regions in data2"""

    data1_ids = np.unique(data1)
    data1_ids = data1_ids[data1_ids > 0].tolist()
    data2_ids = np.unique(data2)
    data2_ids = data2_ids[data2_ids > 0].tolist()

    mapping = {}
    for data1_id in data1_ids:
        region_mask = data1 == data1_id
        (mapping[data1_id],) = mode(data2[region_mask]).mode

    return mapping


def remap_values(original_data, mapping):
    remapped = np.zeros_like(original_data)
    for old_value, new_value in mapping.items():
        remapped[original_data == old_value] = new_value
    return remapped

This code worked in the other notebooks, no need to test here

In [5]:
def add_atlas_to_another(
    atlas1_config,
    atlas1_data,
    atlas2_config,
    atlas2_data,
):
    """Add atlas2 into atlas1.

    Ensure the labels are updated so there is no overlap.
    """
    # Verify inputs
    verify_atlas(atlas1_config, atlas1_data)
    verify_atlas(atlas2_config, atlas2_data)
    atlas2_data = atlas2_data.copy()

    # What is the largest number in atlas1? This will be the minimum
    # value in the new ids for atlas2 after it's been added to atlas1
    merged_atlas_min = np.max(atlas1_data) + 1
    atlas2_id_mapping = dict(
        zip(
            atlas2_config["node_ids"],
            np.argsort(atlas2_config["node_ids"]) + merged_atlas_min,
        ),
    )

    atlas2_merged_ids = [
        atlas2_id_mapping[node_id] for node_id in atlas2_config["node_ids"]
    ]

    # Make sure that any voxels that are labeled in the base image
    # are NOT overwritten by the new atlas data
    atlas2_data[atlas1_data > 0] = 0

    # Ensure that we have not clobbered any regions by doing so
    verify_atlas(atlas2_config, atlas2_data)

    # Change their node ids so they don't conflict with the base
    remapped_atlas2 = remap_values(atlas2_data, atlas2_id_mapping)

    merged_image_data = atlas1_data + remapped_atlas2
    merged_image_labels = atlas1_config["node_names"] + atlas2_config["node_names"]
    merged_image_ids = atlas1_config["node_ids"] + atlas2_merged_ids
    merged_atlas_config = {
        "node_names": merged_image_labels,
        "node_ids": merged_image_ids,
    }
    verify_atlas(merged_atlas_config, merged_image_data)
    return merged_atlas_config, merged_image_data


def expand_df(df):
    spl = df["name"].str.split("_", n=1, expand=True)
    df["label"] = spl[1]
    df["network_id"] = np.nan
    df["network_label"] = np.nan
    df["atlas_name"] = spl[0]
    df.drop("name", axis=1, inplace=True)
    return df

### Creating the full volumetric atlas set: NLin6

The Shaefer NLin6 atlases will get the subcortical regions added to them

In [7]:
nlin6_subcortical_config = tsv_to_config(nlin6_subcortical_tsv)
nlin6_subcortical_data = roi_data(nlin6_subcortical)

for nlin6_schaefer in nlin6_schaefers:
    resolution = nlin6_schaefer.split("desc-")[1].split("Parcels")[0]
    # Get the image and the tsv
    schaefer_data = roi_data(nlin6_schaefer)
    schaefer_config = tsv_to_config(
        f"Schaefer/atlas-Schaefer2018v0143_desc-{resolution}ParcelsAllNetworks_dseg.tsv"
    )
    combined_config_nlin6, combined_data_nlin6 = add_atlas_to_another(
        schaefer_config,
        schaefer_data,
        nlin6_subcortical_config,
        nlin6_subcortical_data,
    )

    subcortical_nlin6 = nb.Nifti1Image(
        combined_data_nlin6, nlin6_subcortical_img.affine
    )
    new_nparcels = int(resolution) + 52
    merged_file_name = f"tpl-MNI152NLin6Asym_atlas-4S{new_nparcels}Parcels_res-01_dseg"
    subcortical_nlin6.to_filename(merged_file_name + ".nii.gz")
    nlin6_df = pd.DataFrame(
        {
            "index": combined_config_nlin6["node_ids"],
            "name": combined_config_nlin6["node_names"],
        },
    )

    # Check this data frame to ensure that the indexes and labels are the same
    official_label_df = pd.read_table(f"atlas-4S{new_nparcels}Parcels_dseg.tsv")
    assert (nlin6_df["index"] == official_label_df["index"]).all()
    assert (nlin6_df["name"] == official_label_df["label"]).all()

PermissionError: [Errno 13] Permission denied: 'tpl-MNI152NLin6Asym_atlas-4S852Parcels_res-01_dseg.nii.gz'

### Creating the full volumetric atlas set: 2009c

The Schaefer 2009c atlases will get the subcortical regions added to them


In [7]:
nlin09c_subcortical_config = tsv_to_config(nlin09c_subcortical_tsv)
nlin09c_subcortical_data = roi_data(nlin09c_subcortical)

for nlin09c_schaefer in nlin09c_schaefers:
    resolution = nlin09c_schaefer.split("desc-")[1].split("Parcels")[0]
    # Get the image and the tsv
    schaefer_data = roi_data(nlin09c_schaefer)
    schaefer_config = tsv_to_config(
        f"Schaefer/atlas-Schaefer2018v0143_desc-{resolution}ParcelsAllNetworks_dseg.tsv"
    )
    combined_config_nlin09c, combined_data_nlin09c = add_atlas_to_another(
        schaefer_config,
        schaefer_data,
        nlin09c_subcortical_config,
        nlin09c_subcortical_data,
    )

    subcortical_nlin09c = nb.Nifti1Image(
        combined_data_nlin09c, nlin09c_subcortical_img.affine
    )
    new_nparcels = int(resolution) + 52
    merged_file_name = (
        f"tpl-MNI152NLin2009cAsym_atlas-4S{new_nparcels}Parcels_res-01_dseg"
    )
    subcortical_nlin09c.to_filename(merged_file_name + ".nii.gz")
    nlin09c_df = pd.DataFrame(
        {
            "index": combined_config_nlin09c["node_ids"],
            "name": combined_config_nlin09c["node_names"],
        },
    )

    # Check this data frame to ensure that the indexes and labels are the same
    official_label_df = pd.read_table(f"atlas-4S{new_nparcels}Parcels_dseg.tsv")
    assert (nlin09c_df["index"] == official_label_df["index"]).all()
    assert (nlin09c_df["name"] == official_label_df["label"]).all()